In [25]:
import torch
import pytorch_lightning as pl
from core.data import GridDataModule
import gif
import matplotlib.pyplot as plt

In [17]:
data_dir = 'data/ignition_center_cut/'
channel = 0

dm = GridDataModule(data_dir=data_dir, dimension=2, batch_size=128, size=25, stride=25, flatten=True, channels=[channel])

In [18]:
class TestModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = torch.nn.Identity()
        
    def forward(self, x):
        return self.model(x)
    
    def predict_step(self, batch, idx):
        return self(batch)
    
    def training_step(self, batch, idx):
        pass
    
    def configure_optimizers(self):
        pass

In [19]:
model = TestModel()
trainer = pl.Trainer(accelerator='gpu', devices=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [20]:
results = trainer.predict(model, datamodule=dm)
data = dm.aglomerate(results)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [23]:
data.shape

torch.Size([450, 50, 50, 1])

In [26]:
@gif.frame
def plot(i):
    plt.imshow(data[i,:,:,channel], vmin=-1, vmax=1, origin='lower')
    plt.colorbar(location='top')

In [27]:
frames = []
for i in range(data.shape[0]):
    frames.append(plot(i))
    
gif.save(frames, f'test.gif', duration=50)